<a href="https://colab.research.google.com/github/shreyasrajesh0308/ece209as_project/blob/main/software/DeepConvLSTMmediummodel_USCHAD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DeepConvLSTM - Medium Model

Building the LSTM architecture and experimenting with the USC HAD dataset

Results are compared for the following cases:

1. Baseline DeepConvLSTM - Medium Model
2. Model with weight decay
3. Model with Rotation. 
4. Model with Scale. 
5. Model with Magnify 
6. Model with Resample
7. Model with Noise

|  Activity |  Baseline |  Weight Decay |  Rotation | Scaling  |  Noise |  Resampling | Magnify | 
|---|---|---|---|---|---|---|---|
| Walking Forward | 0.95  |  0.96 |  0.94 |  0.94 |  0.96 |  0.96 | 0.95 | 
| Walking Left |  0.92 |  0.94 | 0.92 | 0.93  |  0.94 | 0.94  | 0.94 |
| Walking Right  | 0.96  | 0.93  |  0.94 |  0.95 |  0.94 | 0.95  | 0.94 |
| Walking Upstairs  | 0.96  | 0.95  |  0.92 | 0.96  | 0.95  |  0.96 | 0.95 |
| Walking Downstairs | 0.96  |  0.95 |  0.94 |  0.94 | 0.95  |  0.96 | 0.96 |
| Running Forward  |  0.97 |  0.97 |  0.97 | 0.97  | 0.95  |  0.97 | 0.97 |
|  Jumping |  0.96 |  0.97 | 0.96  | 0.94  |  0.97 | 0.96  | 0.97|
|  Sitting |  0.97 |  0.97 | 0.95  | 0.97  |  0.98 | 0.96  | 0.96|
|  Standing |  0.87 |  0.88 | 0.88  | 0.87  |  0.87 | 0.88  | 0.89|
|  Sleeping |  0.99 |  0.99 | 0.96  | 1.0  |  1.0 | 1.0  | 1.0|
|  Elevator Up |  0.57 |  0.48 | 0.51  | 0.44 |  0.5 | 0.53  | 0.51|
|  Elevator Down |  0.34 |  0.44 | 0.46  | 0.54  |  0.51 | 0.47  | 0.47|
|  Overall Accuracy |  0.89 |  0.9 | 0.89  | 0.9  |  0.91 | 0.91  | 0.9|

## Module Import

In [1]:
import tensorflow as tf
from tensorflow import keras 
from keras import layers

import numpy as np

from sklearn.utils import class_weight

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load the Dataset

In [3]:
import os

In [4]:
dataset = '/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/'

In [5]:
import scipy.io
mat = scipy.io.loadmat('/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject13/a11t4.mat')

In [6]:
flag =0
window =128
l = list(range(1,15))
for num in l:
  dataset_address = dataset + "Subject"+str(num) +'/'
  for fil in os.listdir(dataset_address):
    print(str(dataset_address +fil) )
    if str(dataset_address +fil) != '/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject13/a11t4.mat':
      mat = scipy.io.loadmat(str(dataset_address +fil) )
      dat = mat['sensor_readings']
      N = mat['sensor_readings'].shape[0]
      print(N)
      re= N % window
      q = int(N /window)
      dat =dat[ :N-re , :]
      dat = dat.reshape(q,window,6)
      act  = int(mat['activity_number'][0]) -1
      act = q *[act]
      if flag==0:
        X = dat.copy()
        Y = np.array(act)
        flag =1

      else:
        X = np.concatenate((X, dat), axis=0)
        Y = np.concatenate((Y, act), axis=0)

/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a7t1.mat
1100
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a6t1.mat
1500
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a1t1.mat
2400
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a2t1.mat
2600
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a3t1.mat
3100
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a2t2.mat
3000
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a2t3.mat
3600
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a2t4.mat
3500
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a2t5.mat
3700
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a3t2.mat
3900
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a3t3.mat
3800
/content/drive/My Drive/ECE_209_AS_project_data/USC-HAD/Subject1/a3t4.mat
4000
/content/drive/My Drive/ECE_209_AS_project_data/USC-

In [7]:
data_class_names =['walking forward', 'walking left', 'walking right', 'walking upstairs' , 'walking downstairs',  'running forward' ,  'jumping', 'sitting', 'standing ', 'sleeping ',  'elevator up',  'elevator down']
data_class_weights ={}
for i in range(len(data_class_names)+1):
     data_class_weights[i]= 1
print(data_class_weights)

{0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1}


In [8]:
n = int( 0.7 *Y.shape[0])
t = np.random.choice(list(range(Y.shape[0])),n, replace=False)
te = list(set(list(range(Y.shape[0])))-set(t))
X_train = X[t]
Y_train =Y[t]
X_test = X[te]
Y_test =Y[te]

In [9]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(15113, 128, 6)
(15113,)
(6478, 128, 6)
(6478,)


In [10]:
len(data_class_names)

12

### Define the model 

In [11]:
inputs = keras.Input(shape = (128, 6, 1 ))
features = layers.Conv2D(64, kernel_size = (5, 1), strides = (1, 1), padding = "valid", activation = "relu")(inputs)
features = layers.Conv2D(64, kernel_size = (5, 1), strides = (1, 1), padding = "valid", activation = "relu")(features)
#features = layers.Conv2D(64, kernel_size = (5, 1), strides = (1, 1), padding = "valid", activation = "relu")(features)
#features = layers.Conv2D(64, kernel_size = (5, 1), strides = (1, 1), padding = "valid", activation = "relu")(features)
features = layers.Reshape((features.shape[1], features.shape[2]*features.shape[3]))(features)
features = layers.LSTM(128, return_sequences = True, activation = "tanh")(features)
features = layers.Dropout(0.5)(features)
#features = layers.LSTM(128, return_sequences = True, activation = "tanh")(features)
#features = layers.Dropout(0.5)(features)
features = layers.LSTM(128, activation = "tanh")(features)
features = layers.Dropout(0.5)(features)
outputs = layers.Dense(12, activation = "softmax")(features)

model = keras.Model(inputs = inputs, outputs = outputs)


In [12]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 6, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 124, 6, 64)        384       
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 6, 64)        20544     
                                                                 
 reshape (Reshape)           (None, 120, 384)          0         
                                                                 
 lstm (LSTM)                 (None, 120, 128)          262656    
                                                                 
 dropout (Dropout)           (None, 120, 128)          0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584

In [13]:
path_to_logs = "/content/drive/My Drive/ECE_209_AS_project_data/USCHADlogs/"
path_to_models = "/content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/"

### Split the Data and Train the model 

In [14]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [17]:
seeds = []

In [15]:
from sklearn.metrics import confusion_matrix, recall_score

In [18]:
test_accuracies = []
recall_scores = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  train_data = X_train[len(X_train)//10:]
  train_labels = Y_train[len(X_train)//10:]
  val_data = X_train[:len(X_train)//10]
  val_labels = Y_train[:len(X_train)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_baseline_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_baseline_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_baseline_4/assets


tf.Tensor(0.8996604, shape=(), dtype=float32)


In [21]:
# Print Average Total Accuracy 
sum_ac = 0

for ac in test_accuracies:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {sum_ac/len(test_accuracies)}")

Mean accuracy is 0.8957702994346619


In [22]:
mean_recall_scores  = sum(recall_scores)/len(recall_scores)

for i in range(len(mean_recall_scores)):

  print(f"Label {data_class_names[i]} has mean accuracy {mean_recall_scores[i]}")

Label walking forward has mean accuracy 0.952557127312296
Label walking left has mean accuracy 0.9195979899497487
Label walking right has mean accuracy 0.9331058020477816
Label walking upstairs has mean accuracy 0.9641137855579869
Label walking downstairs has mean accuracy 0.9584650112866816
Label running forward has mean accuracy 0.9672131147540982
Label jumping has mean accuracy 0.9576923076923076
Label sitting has mean accuracy 0.9725888324873097
Label standing  has mean accuracy 0.872822299651568
Label sleeping  has mean accuracy 0.9988876529477195
Label elevator up has mean accuracy 0.576271186440678
Label elevator down has mean accuracy 0.34123989218328843


In [23]:
import matplotlib.pyplot as plt

## Same model with regularization

In [24]:
from tensorflow.keras import regularizers

In [25]:
test_accuracies_with_reg = []
recall_scores_with_reg = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  train_data = X_train[len(X_train)//10:]
  train_labels = Y_train[len(X_train)//10:]
  val_data = X_train[:len(X_train)//10]
  val_labels = Y_train[:len(X_train)//10]

  model_with_reg = keras.Model(inputs = inputs, outputs = outputs)
  model_with_reg.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_with_reg_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model_with_reg.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_with_reg_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_with_reg.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_with_reg.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_with_reg_4/assets


tf.Tensor(0.89935166, shape=(), dtype=float32)


In [26]:
sum_ac = 0

for ac in test_accuracies_with_reg:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {sum_ac/len(test_accuracies_with_reg)}")

Mean accuracy is 0.8985180616378784


In [27]:
mean_recall_scores_with_reg  = sum(recall_scores_with_reg)/len(recall_scores_with_reg)

for i in range(len(mean_recall_scores_with_reg)):

  print(f"Label {data_class_names[i]} has mean accuracy {mean_recall_scores_with_reg[i]}")

Label walking forward has mean accuracy 0.9597388465723613
Label walking left has mean accuracy 0.9403685092127304
Label walking right has mean accuracy 0.9337883959044369
Label walking upstairs has mean accuracy 0.9527352297592998
Label walking downstairs has mean accuracy 0.9512415349887133
Label running forward has mean accuracy 0.9695550351288056
Label jumping has mean accuracy 0.9653846153846155
Label sitting has mean accuracy 0.966497461928934
Label standing  has mean accuracy 0.8801393728222996
Label sleeping  has mean accuracy 0.999777530589544
Label elevator up has mean accuracy 0.4836158192090395
Label elevator down has mean accuracy 0.4361185983827493


## With Augmentations

1. Rotation
2. Scale
3. Magnify
4. Resample
5. Noise

### Augmentations

In [28]:
import numpy as np
import tensorflow as tf
import random

def resampling_fast_random(x):
    M, N = random.choice([[1, 0], [2, 1], [3, 2]])
    time_steps = x.shape[1]
    raw_set = np.arange(x.shape[1])
    interp_steps = np.arange(0, raw_set[-1] + 1e-1, 1 / (M + 1))
    x_interp = interp1d(raw_set, x, axis=1)
    x_up = x_interp(interp_steps)

    length_inserted = x_up.shape[1]
    start = random.randint(0, length_inserted - time_steps * (N + 1))
    index_selected = np.arange(start, start + time_steps * (N + 1), N + 1)
    return x_up[:, index_selected, :]

def resampling(x,M,N):
    '''
    :param x: the data of a batch,shape=(batch_size,timesteps,features)
    :param M: the number of  new value under tow values
    :param N: the interval of resampling
    :return: x after resampling，shape=(batch_size,timesteps,features)
    '''
    assert M>N,'the value of M have to greater than N'

    timesetps = x.shape[1]

    for i in range(timesetps-1):
        x1 = x[:,i*(M+1),:]
        x2 = x[:,i*(M+1)+1,:]
        for j in range(M):
            v = np.add(1,np.subtract(x2,x1)*(j+1)/(M+1))
            x = np.insert(x,i*(M+1)+j+1,v,axis=1)

    length_inserted = x.shape[1]
    start = random.randint(0,length_inserted-timesetps*(N+1))
    index_selected = np.arange(start,start+timesetps*(N+1),N+1)
    return x[:,index_selected,:]
    return x

def noise(x):
    x = tf.add(x,tf.multiply(x,tf.cast(tf.random.uniform(shape = (x.shape[0],x.shape[1],x.shape[2]),minval=-0.1,maxval=0.1),tf.float64)))
    return x


def scaling(x):
    alpha = np.random.randint(7,10)/10
    # alpha = 0.9
    return tf.multiply(x,alpha)
#
def magnify(x):
    lam = np.random.randint(11,14)/10
    return tf.multiply(x,lam)


def inverting(x):
    return np.multiply(x,-1)
def reversing(x):
    return x[:,-1::-1,:]


def rotation(x):
    c = x.shape[2]//3
    x_new = rotation_transform_vectorized(x[:,:,0:3])
    for i in range(1,c):
        temp = rotation_transform_vectorized(x[:,:,i*3:(i+1)*3])
        x_new = np.concatenate((x_new,temp),axis=-1)
    return x_new
def rotation_transform_vectorized(X):
    """
    Applying a random 3D rotation
    """
    axes = np.random.uniform(low=-1, high=1, size=(X.shape[0], X.shape[2]))
    angles = np.random.uniform(low=-np.pi, high=np.pi, size=(X.shape[0]))
    matrices = axis_angle_to_rotation_matrix_3d_vectorized(axes, angles)

    return np.matmul(X, matrices)
def axis_angle_to_rotation_matrix_3d_vectorized(axes, angles):
    """
    Get the rotational matrix corresponding to a rotation of (angle) radian around the axes
    Reference: the Transforms3d package - transforms3d.axangles.axangle2mat
    Formula: http://en.wikipedia.org/wiki/Rotation_matrix#Axis_and_angle
    """
    axes = axes / np.linalg.norm(axes, ord=2, axis=1, keepdims=True)
    x = axes[:, 0]; y = axes[:, 1]; z = axes[:, 2]
    c = np.cos(angles)
    s = np.sin(angles)
    C = 1 - c

    xs = x*s;   ys = y*s;   zs = z*s
    xC = x*C;   yC = y*C;   zC = z*C
    xyC = x*yC; yzC = y*zC; zxC = z*xC

    m = np.array([
        [ x*xC+c,   xyC-zs,   zxC+ys ],
        [ xyC+zs,   y*yC+c,   yzC-xs ],
        [ zxC-ys,   yzC+xs,   z*zC+c ]])
    matrix_transposed = np.transpose(m, axes=(2,0,1))
    return matrix_transposed
## This example using cubic splice is not the best approach to generate random curves. 
## You can use other aprroaches, e.g., Gaussian process regression, Bezier curve, etc.
def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
    x_range = np.arange(X.shape[0])
    cs_x = CubicSpline(xx[:,0], yy[:,0])
    cs_y = CubicSpline(xx[:,1], yy[:,1])
    cs_z = CubicSpline(xx[:,2], yy[:,2])
    return np.array([cs_x(x_range),cs_y(x_range),cs_z(x_range)]).transpose()

  
def DistortTimesteps(X, sigma=0.2):
    tt = GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
    tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = [(X.shape[0]-1)/tt_cum[-1,0],(X.shape[0]-1)/tt_cum[-1,1],(X.shape[0]-1)/tt_cum[-1,2]]
    tt_cum[:,0] = tt_cum[:,0]*t_scale[0]
    tt_cum[:,1] = tt_cum[:,1]*t_scale[1]
    tt_cum[:,2] = tt_cum[:,2]*t_scale[2]
    return tt_cum

def DA_TimeWarp(X, sigma=0.2):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])
    X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
    X_new[:,1] = np.interp(x_range, tt_new[:,1], X[:,1])
    X_new[:,2] = np.interp(x_range, tt_new[:,2], X[:,2])
    return X_new


### Rotation

In [29]:
test_accuracies_rot = []
recall_scores_rot = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_rot = rotation((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_rot))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_withrot_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_withrot_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_rot.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_rot.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withrot_4/assets


tf.Tensor(0.8887002, shape=(), dtype=float32)


In [30]:
sum_ac = 0

for ac in test_accuracies_rot:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_rot), 2)}")

Mean accuracy is 0.89


In [31]:
mean_recall_scores_with_rot  = sum(recall_scores_rot)/len(recall_scores_rot)

for i in range(len(mean_recall_scores_with_rot)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_rot[i], 2)}")

Label walking forward has mean accuracy 0.94
Label walking left has mean accuracy 0.92
Label walking right has mean accuracy 0.94
Label walking upstairs has mean accuracy 0.92
Label walking downstairs has mean accuracy 0.94
Label running forward has mean accuracy 0.97
Label jumping has mean accuracy 0.96
Label sitting has mean accuracy 0.95
Label standing  has mean accuracy 0.88
Label sleeping  has mean accuracy 0.96
Label elevator up has mean accuracy 0.51
Label elevator down has mean accuracy 0.46


### Scaling

In [32]:
test_accuracies_scale = []
recall_scores_scale = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_scale = scaling((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_scale))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_withscale_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_withscale_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_scale.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_scale.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withscale_4/assets


tf.Tensor(0.896573, shape=(), dtype=float32)


In [33]:
sum_ac = 0

for ac in test_accuracies_scale:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_scale), 2)}")

Mean accuracy is 0.9


In [34]:
mean_recall_scores_with_scale  = sum(recall_scores_scale)/len(recall_scores_scale)

for i in range(len(mean_recall_scores_with_scale)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_scale[i], 2)}")

Label walking forward has mean accuracy 0.94
Label walking left has mean accuracy 0.93
Label walking right has mean accuracy 0.95
Label walking upstairs has mean accuracy 0.96
Label walking downstairs has mean accuracy 0.94
Label running forward has mean accuracy 0.97
Label jumping has mean accuracy 0.94
Label sitting has mean accuracy 0.97
Label standing  has mean accuracy 0.87
Label sleeping  has mean accuracy 1.0
Label elevator up has mean accuracy 0.44
Label elevator down has mean accuracy 0.54


### Magnify

In [35]:
test_accuracies_magnify = []
recall_scores_magnify = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_magnify = magnify((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_magnify))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_withmag_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_withmag_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_magnify.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_magnify.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withmag_4/assets


tf.Tensor(0.9079963, shape=(), dtype=float32)


In [36]:
sum_ac = 0

for ac in test_accuracies_magnify:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_magnify), 2)}")

Mean accuracy is 0.9


In [37]:
mean_recall_scores_with_magnify  = sum(recall_scores_magnify)/len(recall_scores_magnify)

for i in range(len(mean_recall_scores_with_magnify)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_magnify[i], 2)}")

Label walking forward has mean accuracy 0.95
Label walking left has mean accuracy 0.94
Label walking right has mean accuracy 0.94
Label walking upstairs has mean accuracy 0.95
Label walking downstairs has mean accuracy 0.96
Label running forward has mean accuracy 0.97
Label jumping has mean accuracy 0.97
Label sitting has mean accuracy 0.96
Label standing  has mean accuracy 0.89
Label sleeping  has mean accuracy 1.0
Label elevator up has mean accuracy 0.51
Label elevator down has mean accuracy 0.47


### Resampling

In [38]:
from sklearn.metrics import recall_score

In [39]:
test_accuracies_resampling = []
recall_scores_resampling = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_resample = resampling((X_train), 1, 0)
  Xtrain_tot=np.concatenate((X_train, X_train_resample))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_withresampling_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_withresampling_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_resampling.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_resampling.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withresampling_4/assets


tf.Tensor(0.90861374, shape=(), dtype=float32)


In [40]:
sum_ac = 0

for ac in test_accuracies_resampling:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_resampling), 2)}")

Mean accuracy is 0.91


In [41]:
mean_recall_scores_with_resampling  = sum(recall_scores_resampling)/len(recall_scores_resampling)

for i in range(len(mean_recall_scores_with_resampling)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_resampling[i], 2)}")

Label walking forward has mean accuracy 0.96
Label walking left has mean accuracy 0.94
Label walking right has mean accuracy 0.95
Label walking upstairs has mean accuracy 0.96
Label walking downstairs has mean accuracy 0.96
Label running forward has mean accuracy 0.97
Label jumping has mean accuracy 0.96
Label sitting has mean accuracy 0.96
Label standing  has mean accuracy 0.88
Label sleeping  has mean accuracy 1.0
Label elevator up has mean accuracy 0.53
Label elevator down has mean accuracy 0.47


### Noise 

In [42]:
test_accuracies_noise = []
recall_scores_noise = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_noise = noise((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_noise))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"DeepConvLSTMmediumUSCHAD_withnoise_{i}"), tensorboard, keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]
  history = model.fit(train_data, train_labels, epochs = 100, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"DeepConvLSTMmediumUSCHAD_withnoise_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_noise.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_noise.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/USCHADmodels/DeepConvLSTMmediumUSCHAD_withnoise_4/assets


tf.Tensor(0.8998148, shape=(), dtype=float32)


In [45]:
sum_ac = 0

for ac in test_accuracies_noise:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_noise), 2)}")

Mean accuracy is 0.91


In [46]:
mean_recall_scores_with_noise  = sum(recall_scores_noise)/len(recall_scores_noise)

for i in range(len(mean_recall_scores_with_noise)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_noise[i], 2)}")

Label walking forward has mean accuracy 0.96
Label walking left has mean accuracy 0.94
Label walking right has mean accuracy 0.94
Label walking upstairs has mean accuracy 0.95
Label walking downstairs has mean accuracy 0.95
Label running forward has mean accuracy 0.97
Label jumping has mean accuracy 0.97
Label sitting has mean accuracy 0.98
Label standing  has mean accuracy 0.87
Label sleeping  has mean accuracy 1.0
Label elevator up has mean accuracy 0.5
Label elevator down has mean accuracy 0.51


## Results

|  Activity |  Baseline |  Weight Decay |  Rotation | Scaling  |  Noise |  Resampling | Magnify | 
|---|---|---|---|---|---|---|---|
| Walking  | 0.94  |  0.95 |  0.96 |  0.96 |  0.95 |  0.94 | 0.97 | 
| Stairs-Up  |  0.96 |  0.97 | 0.97 | 0.98  |  0.95 | 0.96  | 0.98 |
| Stairs-Down  | .99  | 0.99  |  0.99 |  0.99 |  0.99 | 1.0  | 0.99 |
| Sitting  | 0.80  | 0.77  |  0.8 | 0.82  | 0.79  |  0.79 | 0.81 |
| Standing | 0.83  |  0.86 |  0.79 |  0.9 | 0.81  |  0.83 | 0.94 |
| Lying  |  1 |  0.99 |  0.94 | 1  | 1  |  1 | 1 |
|  Overall Accuracy |  0.92 |  0.92 | 0.9  | 0.94  |  0.91 | 0.92  | 0.95|